In [81]:
import pandas as pd

In [82]:
import sqlite3

In [83]:
bookings = pd.read_csv('bookings.csv')
facilities = pd.read_csv('facilities.csv')
members = pd.read_csv('members.csv')

In [84]:
bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [85]:
facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [86]:
members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,NaN,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,NaN,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,NaN,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,NaN,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1.0,2012-07-03 10:25:05


In [87]:
db = sqlite3.connect('country_club_db')

In [88]:
bookings.to_sql('book',db, if_exists='replace')

In [89]:
facilities.to_sql('fac',db, if_exists='replace')

In [90]:
members.to_sql('mem', db, if_exists='replace')

In [91]:
db

## /*Q1: Some of the facilities charge a fee to members, but some do not.
## Please list the names of the facilities that do. */

In [92]:
pd.read_sql_query('SELECT * FROM fac WHERE membercost != 0', db)

,index,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,1,Tennis Court 2,5.0,25.0,8000,200
2,4,4,Massage Room 1,9.9,80.0,4000,3000
3,5,5,Massage Room 2,9.9,80.0,4000,3000
4,6,6,Squash Court,3.5,17.5,5000,80


## /* Q2: How many facilities do not charge a fee to members? */

In [93]:
pd.read_sql_query('SELECT count(*) FROM fac WHERE membercost = 0', db)

,count(*)
0,4


## /* Q3: How can you produce a list of facilities that charge a fee to members,
## where the fee is less than 20% of the facility's monthly maintenance cost?
## Return the facid, facility name, member cost, and monthly maintenance of the
## facilities in question. */

In [94]:
pd.read_sql_query('SELECT facid, name, membercost, monthlymaintenance, monthlymaintenance*.2 AS twenty_per FROM fac \
WHERE membercost < (monthlymaintenance*0.2)', db)

,facid,name,membercost,monthlymaintenance,twenty_per
0,0,Tennis Court 1,5.0,200,40.0
1,1,Tennis Court 2,5.0,200,40.0
2,2,Badminton Court,0.0,50,10.0
3,3,Table Tennis,0.0,10,2.0
4,4,Massage Room 1,9.9,3000,600.0
5,5,Massage Room 2,9.9,3000,600.0
6,6,Squash Court,3.5,80,16.0
7,7,Snooker Table,0.0,15,3.0
8,8,Pool Table,0.0,15,3.0


## /* Q4: How can you retrieve the details of facilities with ID 1 and 5?
## Write the query without using the OR operator. */

In [95]:
pd.read_sql_query('SELECT * FROM fac WHERE facid in(1,5)', db)

,index,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,1,Tennis Court 2,5.0,25.0,8000,200
1,5,5,Massage Room 2,9.9,80.0,4000,3000


## /* Q5: How can you produce a list of facilities, with each labelled as
## 'cheap' or 'expensive', depending on if their monthly maintenance cost is
## more than $100? Return the name and monthly maintenance of the facilities
## in question. */

In [96]:
pd.read_sql_query("SELECT name, monthlymaintenance,CASE WHEN monthlymaintenance > 100 THEN 'expensive'\
WHEN monthlymaintenance <= 100 THEN 'cheap'  ELSE NULL END AS costtype FROM fac", db)

,name,monthlymaintenance,costtype
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


## /* Q6: You'd like to get the first and last name of the last member(s)
## who signed up. Do not use the LIMIT clause for your solution. */

In [97]:
pd.read_sql_query('SELECT * FROM mem WHERE \
joindate = (SELECT MAX(joindate) from mem)', db)

,index,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,30,37,Smith,Darren,"3 Funktown, Denzington, Boston",66796,(822) 577-3541,None,2012-09-26 18:08:45


## /* Q7: How can you produce a list of all members who have used a tennis court?
## Include in your output the name of the court, and the name of the member
## formatted as a single column. Ensure no duplicate data, and order by
## the member name. */

In [98]:
pd.read_sql_query('SELECT b.memid, b.facid, (m.firstname ||" "|| m.surname) AS member_name, f.name AS facility_name \
FROM book b \
JOIN fac f ON f.facid = b.facid \
JOIN mem m ON m.memid = b.memid \
WHERE b.facid IN (0,1) GROUP BY 1,2,3,4 ORDER BY 3', db)

,memid,facid,member_name,facility_name
0,12,0,Anne Baker,Tennis Court 1
1,12,1,Anne Baker,Tennis Court 2
2,6,0,Burton Tracy,Tennis Court 1
3,6,1,Burton Tracy,Tennis Court 2
4,10,0,Charles Owen,Tennis Court 1
5,10,1,Charles Owen,Tennis Court 2
6,1,1,Darren Smith,Tennis Court 2
7,28,0,David Farrell,Tennis Court 1
8,28,1,David Farrell,Tennis Court 2
9,11,0,David Jones,Tennis Court 1


## /* Q8: How can you produce a list of bookings on the day of 2012-09-14 which
## will cost the member (or guest) more than $30? Remember that guests have
## different costs to members (the listed costs are per half-hour 'slot'), and
## the guest user's ID is always 0. Include in your output the name of the
## facility, the name of the member formatted as a single column, and the cost.
## Order by descending cost, and do not use any subqueries. */

In [99]:
pd.read_sql_query('SELECT DATE(b.starttime) AS dt, b.memid, b.facid, (m.firstname ||" "|| m.surname) AS member_name, \
f.name AS facility_name, \
CASE WHEN b.memid = 0 THEN (b.slots*f.guestcost) \
ELSE (b.slots*f.membercost) END AS total_cost FROM book b \
JOIN fac f ON f.facid = b.facid \
JOIN mem m ON m.memid = b.memid \
WHERE DATE(b.starttime) = DATE("2012-09-14") AND total_cost > 30 GROUP BY 1,2,3,4,5,6 ORDER BY 6 DESC', db)

,dt,memid,facid,member_name,facility_name,total_cost
0,2012-09-14,0,5,GUEST GUEST,Massage Room 2,320.0
1,2012-09-14,0,4,GUEST GUEST,Massage Room 1,160.0
2,2012-09-14,0,1,GUEST GUEST,Tennis Court 2,150.0
3,2012-09-14,0,0,GUEST GUEST,Tennis Court 1,75.0
4,2012-09-14,0,1,GUEST GUEST,Tennis Court 2,75.0
5,2012-09-14,0,6,GUEST GUEST,Squash Court,70.0
6,2012-09-14,13,4,Jemima Farrell,Massage Room 1,39.6
7,2012-09-14,0,6,GUEST GUEST,Squash Court,35.0


## /* Q9: This time, produce the same result as in Q8, but using a subquery. */

In [100]:
pd.read_sql_query('select date(b.starttime) AS dt, b.memid, b.facid, \
(select (m.firstname ||" "|| m.surname) AS member_name from mem m where m.memid = b.memid)member_name, \
(select f.name from fac f where f.facid = b.facid)facility_name, \
(select CASE WHEN b.memid = 0 THEN (b.slots*f.guestcost) WHEN b.memid > 0 THEN (b.slots*f.membercost) ELSE NULL \
END AS total_cost from fac f where f.facid = b.facid)total_cost \
from book b where date(b.starttime)=date("2012-09-14") AND total_cost > 30 group by 1,2,3,4,5,6 order by 6 DESC',db)

,dt,memid,facid,member_name,facility_name,total_cost
0,2012-09-14,0,5,GUEST GUEST,Massage Room 2,320.0
1,2012-09-14,0,4,GUEST GUEST,Massage Room 1,160.0
2,2012-09-14,0,1,GUEST GUEST,Tennis Court 2,150.0
3,2012-09-14,0,0,GUEST GUEST,Tennis Court 1,75.0
4,2012-09-14,0,1,GUEST GUEST,Tennis Court 2,75.0
5,2012-09-14,0,6,GUEST GUEST,Squash Court,70.0
6,2012-09-14,13,4,Jemima Farrell,Massage Room 1,39.6
7,2012-09-14,0,6,GUEST GUEST,Squash Court,35.0


## /* Q10: Produce a list of facilities with a total revenue less than 1000.
## The output of facility name and total revenue, sorted by revenue. Remember
## that there's a different cost for guests and members! */

In [101]:
pd.read_sql_query('SELECT f.facid, f.name AS facility_name, \
(SELECT SUM(CASE WHEN b.memid = 0 THEN (b.slots*f.guestcost) \
ELSE (b.slots*f.membercost) END) FROM book b \
WHERE f.facid = b.facid)total_revenue FROM fac f WHERE total_revenue < 1000 \
GROUP BY 1,2 ORDER BY 3', db)

,facid,facility_name,total_revenue
0,3,Table Tennis,180.0
1,7,Snooker Table,240.0
2,8,Pool Table,270.0
